In [4]:
import pandas as pd
import numpy as np
df=pd.read_csv("loan.csv")
df.head(5)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Normalizer

In [6]:
df.corr()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
ApplicantIncome,1.000000,-0.116605,0.570909,-0.045306,-0.014715
CoapplicantIncome,-0.116605,1.000000,0.188619,-0.059878,-0.002056
LoanAmount,0.570909,0.188619,1.000000,0.039447,-0.008433
Loan_Amount_Term,-0.045306,-0.059878,0.039447,1.000000,0.001470
Credit_History,-0.014715,-0.002056,-0.008433,0.001470,1.000000


In [7]:
df.drop(['Loan_ID','Gender','Married','Dependents','CoapplicantIncome'],axis=1,inplace=True)
df

,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Graduate,No,5849,NaN,360.0,1.0,Urban,Y
1,Graduate,No,4583,128.0,360.0,1.0,Rural,N
2,Graduate,Yes,3000,66.0,360.0,1.0,Urban,Y
3,Not Graduate,No,2583,120.0,360.0,1.0,Urban,Y
4,Graduate,No,6000,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...
609,Graduate,No,2900,71.0,360.0,1.0,Rural,Y
610,Graduate,No,4106,40.0,180.0,1.0,Rural,Y
611,Graduate,No,8072,253.0,360.0,1.0,Urban,Y
612,Graduate,No,7583,187.0,360.0,1.0,Urban,Y


In [8]:

df.isnull().sum()

Education            0
Self_Employed       32
ApplicantIncome      0
LoanAmount          22
Loan_Amount_Term    14
Credit_History      50
Property_Area        0
Loan_Status          0
dtype: int64

In [9]:

df['Self_Employed'].fillna(method='ffill',inplace=True)

In [10]:
df['LoanAmount'].fillna(df['LoanAmount'].mean(),inplace=True)

In [11]:
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mean(),inplace=True)

In [12]:
df['Credit_History'].fillna(method='ffill',inplace=True)

In [13]:

df.isnull().sum()

Education           0
Self_Employed       0
ApplicantIncome     0
LoanAmount          0
Loan_Amount_Term    0
Credit_History      0
Property_Area       0
Loan_Status         0
dtype: int64

In [14]:
df.shape

(614, 8)

In [15]:

Education = pd.get_dummies(df['Education'],drop_first=True)

In [16]:
Self_Employed = pd.get_dummies(df['Self_Employed'],drop_first=True)

In [17]:
Property_Area = pd.get_dummies(df['Property_Area'],drop_first=True)

In [18]:
Loan_Status =pd.get_dummies(df['Loan_Status'],drop_first=True)

In [19]:

mod_df = pd.concat([df,Education,Self_Employed,Property_Area,Loan_Status],axis=1)

In [20]:
mod_df.head(2)

,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Not Graduate,Yes,Semiurban,Urban,Y
0,Graduate,No,5849,146.412162,360.0,1.0,Urban,Y,0,0,0,1,1
1,Graduate,No,4583,128.000000,360.0,1.0,Rural,N,0,0,0,0,0


In [21]:
mod_df.drop(["Education","Self_Employed","Property_Area","Loan_Status"],axis = 1 ,inplace = True)

In [22]:
mod_df

,ApplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Not Graduate,Yes,Semiurban,Urban,Y
0,5849,146.412162,360.0,1.0,0,0,0,1,1
1,4583,128.000000,360.0,1.0,0,0,0,0,0
2,3000,66.000000,360.0,1.0,0,1,0,1,1
3,2583,120.000000,360.0,1.0,1,0,0,1,1
4,6000,141.000000,360.0,1.0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...
609,2900,71.000000,360.0,1.0,0,0,0,0,1
610,4106,40.000000,180.0,1.0,0,0,0,0,1
611,8072,253.000000,360.0,1.0,0,0,0,1,1
612,7583,187.000000,360.0,1.0,0,0,0,1,1


In [23]:
#s8:check for null values in the new dataframe
mod_df.isnull().sum()

ApplicantIncome     0
LoanAmount          0
Loan_Amount_Term    0
Credit_History      0
Not Graduate        0
Yes                 0
Semiurban           0
Urban               0
Y                   0
dtype: int64

In [24]:
mod_df.dtypes

ApplicantIncome       int64
LoanAmount          float64
Loan_Amount_Term    float64
Credit_History      float64
Not Graduate          uint8
Yes                   uint8
Semiurban             uint8
Urban                 uint8
Y                     uint8
dtype: object

In [25]:
mod_df.shape

(614, 9)

In [26]:
X = mod_df.iloc[:,0:8]
Y = mod_df.iloc[:,8]

In [27]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
rescaledX = scaler.transform(X)

In [28]:
print(type(rescaledX))

<class 'numpy.ndarray'>


In [29]:
from sklearn.preprocessing import Normalizer
scaler = Normalizer().fit(rescaledX)
normalizedX = scaler.transform(rescaledX)

In [30]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(normalizedX)

In [31]:
print(rescaledX)
#print(Y)

[[0.38494237 0.41993543 0.61123126 ... 0.06612624 0.11368605 1.        ]
 [0.29291981 0.31713929 0.63985255 ... 0.00344507 0.00592286 0.00546626]
 [0.27680543 0.2289496  0.57471458 ... 0.90015024 0.25117652 0.74119203]
 ...
 [0.48691554 0.77925603 0.59493913 ... 0.10180632 0.17502824 0.88453128]
 [0.48199348 0.58104556 0.60708757 ... 0.075201   0.12928764 0.97063204]
 [0.33413261 0.39220376 0.56847209 ... 0.81621561 0.69123782 0.25350519]]


In [32]:
print(type(rescaledX))

<class 'numpy.ndarray'>


In [33]:
print(Y)

0      1
1      0
2      1
3      1
4      1
      ..
609    1
610    1
611    1
612    1
613    0
Name: Y, Length: 614, dtype: uint8


In [34]:
X_train,X_test,Y_train,Y_test = train_test_split(rescaledX,Y,random_state=0,test_size=0.2)

In [35]:
len(Y_test)

123

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(rescaledX, Y, test_size=0.30, random_state=42, stratify = Y)
maxK = 15
accuracies = []
for k in range(1,maxK+1):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, Y_train)
    acc = knn.score(X_test, Y_test)*100
    print('For k =',k, 'Accuracy = ', acc, '%')
    accuracies.append(acc)

In [ ]:
import math
math.sqrt(len(Y_test))

In [ ]:
len(Y_test)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=11, p=2,metric='euclidean')

In [ ]:
classifier.fit(X_train, Y_train)

In [ ]:
Y_pred = classifier.predict(X_test)

In [ ]:
#print(len(Y_pred))
#print(len(Y_train))
#print(Y_test)
print(len(Y_test))
#print(Y_pred)

In [ ]:
print(f1_score(Y_test, Y_pred))

In [ ]:
print(accuracy_score(Y_test,Y_pred))

In [ ]:
classifier.predict([[5849,146.412162,360.0,1.0,0,0,0,1]])

In [ ]:
classifier.predict([[4583,128.000000,360.0,1.0,0,0,0,0]])

In [ ]:
##predicting using logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

In [ ]:
y_predicted = model.predict(X_test)

In [ ]:
print(f1_score(Y_test, y_predicted))

In [ ]:
model.score(X_test,Y_test)

In [ ]:
###applying the bayes 

In [ ]:
# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, Y_train)


In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)


In [ ]:
cm

In [ ]:
print(f1_score(Y_test, y_pred))